In [395]:
import json
import spacy
import en_core_web_lg
nlp = spacy.load("en_core_web_lg")
import string
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, make_scorer
from random import shuffle
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
import eli5
from langdetect import detect
import re
from spacy.tokenizer import Tokenizer
from spacy.lang.en.tokenizer_exceptions import TOKENIZER_EXCEPTIONS

# Дані : "Улліс" Джейма Джойса

In [2]:
with open('4300-0.txt') as f:
    content = f.readlines()
content[:15]

['\ufeff\n',
 'The Project Gutenberg EBook of Ulysses, by James Joyce\n',
 '\n',
 'This eBook is for the use of anyone anywhere at no cost and with\n',
 'almost no restrictions whatsoever. You may copy it, give it away or\n',
 're-use it under the terms of the Project Gutenberg License included\n',
 'with this eBook or online at www.gutenberg.org\n',
 '\n',
 '\n',
 'Title: Ulysses\n',
 '\n',
 'Author: James Joyce\n',
 '\n',
 'Release Date: August 1, 2008 [EBook #4300]\n',
 'Last Updated: December 27, 2019\n']

In [3]:
with open('4300-0.txt') as f:
    content = f.read()
text = content.split("— I —\n\n\n[ 1 ]")[1]

In [4]:
text = text.replace('\n', ' ').replace('—', '').replace('_', '').replace('’',"'")
t1 = text[:1000000]
sents1 = []
doc = nlp(t1)
for sent in doc.sents:
    if sent.string.strip()[-1] in ['!', '.', '?'] and sent.string.strip()[0].istitle():
        sents1.append(sent.string.strip())
t2 = text[1000000:]
sents2 = []
doc = nlp(t2)
for sent in doc.sents:
    if sent.string.strip()[-1] in ['!', '.', '?'] and sent.string.strip()[0].istitle():
        sents2.append(sent.string.strip())
sents = sents1 + sents2

In [5]:
len(sents)

21168

In [6]:
sents[0]

'Stately, plump Buck Mulligan came from the stairhead, bearing a bowl of lather on which a mirror and a razor lay crossed.'

In [7]:
# У тексті багато мов, залишаю речення англійською
en = []
for sent in sents:
    if detect(sent)=='en':
        en.append(sent)
len(en)/len(sents)

0.8013510959939532

# Формую датасет

In [8]:
# sent1 - буде 1 речення, sent2 - 2 склеєні речення і так далі
sent2 = en[: int(len(en) * .70)]
sent3 = en[int(len(en) * .70):int(len(en) * .95)]
sent1 = en[int(len(en) * .95):int(len(en) * .975)]
sent4 = en[int(len(en) * .975):]

In [70]:
def decapitalize(text):
    if len(text.split(' ')[0])>1:
        res = [text.split(' ')[0].lower()]+text.split(' ')[1:]
    else:
        res = [text.split(' ')[0]]+text.split(' ')[1:]
    return ' '.join(res)

In [74]:
def depunctuate(text):
    res = text.split(' ')[:-1]+[text.split(' ')[-1].strip(string.punctuation)]
    return ' '.join(res)

In [75]:
def label(text):
    sent = []
    doc = nlp(text)
    for token in doc:
        if token in doc[:-1]:
            sent.append([token.text, False])
        else:
            sent.append([token.text, True])
    return sent

In [76]:
def label_p(text):
    sent = []
    doc = nlp(text)
    for token in doc:
        sent.append([token.text, False])
    return sent

In [77]:
dsent2 = [(decapitalize(x) if i % 4 == 1 else x) for i, x in enumerate(sent2)]
dp2 = [(depunctuate(x) if i%2==0 else x) for i, x in enumerate(dsent2)]
lab2 = [(label(x) if i%2==0 else label_p(x)) for i, x in enumerate(dp2)]
labeled_2 = [x+y for x,y in zip(*[iter(lab2)]*2)]

In [78]:
dsent3 = [(decapitalize(x) if i % 5 == 0 else x) for i, x in enumerate(sent3)]
dp3 = [(x if i in range(2, len(dsent3)+1, 3) else depunctuate(x)) for i, x in enumerate(dsent3)]
lab3 = [(label_p(x) if i in range(2, len(dsent3)+1, 3) else label(x)) for i, x in enumerate(dp3)]
labeled_3 = [x+y+z for x,y,z in zip(*[iter(lab3)]*3)]

In [79]:
dsent4 = [(decapitalize(x) if i % 8 == 1 else x) for i, x in enumerate(sent4)]
dp4 = [(x if i in range(3, len(dsent4)+1, 4) else depunctuate(x)) for i, x in enumerate(dsent4)]
lab4 = [(label_p(x) if i in range(3, len(dsent4)+1, 4) else label(x)) for i, x in enumerate(dp4)]
labeled_4 = [x+y+z+e for x,y,z,e in zip(*[iter(lab4)]*4)]

In [80]:
labeled_1 = [label_p(x) for x in sent1]

In [81]:
data = labeled_1 + labeled_2+ labeled_3+labeled_3
shuffle(data)
len(data)

9187

In [82]:
data[0]

[['Could', False],
 ['have', False],
 ['given', False],
 ['that', False],
 ['address', False],
 ['too', True],
 ['And', False],
 ['past', False],
 ['the', False],
 ['sailors', False],
 ["'", False],
 ['home', False],
 ['.', False]]

In [83]:
def custom_tokenizer(nlp):
    hyphen_re = re.compile(r'[A-Za-z\d]+-[A-Za-z\d]+')
    
    prefixes = list(nlp.Defaults.prefixes)
    prefixes.remove('"')
    prefix_re = spacy.util.compile_prefix_regex(prefixes)
    infix_re = spacy.util.compile_infix_regex(nlp.Defaults.infixes)
    suffixes = list(nlp.Defaults.suffixes)
    suffixes.remove('"')
    suffix_re = spacy.util.compile_suffix_regex(suffixes)

    return Tokenizer(nlp.vocab, rules=TOKENIZER_EXCEPTIONS, prefix_search=prefix_re.search, infix_finditer=infix_re.finditer,
                         suffix_search=suffix_re.search, token_match=hyphen_re.match)

nlp.tokenizer = custom_tokenizer(nlp)
contracted_verbs = ['’m','’re','’s','’t','’ve','’d','’ll','n’t',"'m","'re","'s","'t","'ve","'d","'ll","n't"]
def sent2features(sent):
    tokens = []
    for i in range(len(sent)):
        tokens.append(sent[i][0])
    text = ''.join([('' if c in contracted_verbs else ' ')+c for c in tokens]).strip()
    doc = nlp(text)
    res = [(token.text, token.pos_, token.dep_, token.head.text, token.head.pos_, doc.get_lca_matrix()[i], token.n_lefts, token.n_rights) for i, token in enumerate(doc)]
    return res

In [84]:
sent2features(data[0])[0]

('Could',
 'VERB',
 'aux',
 'given',
 'VERB',
 array([ 0,  2,  2,  2,  2,  2, -1, -1, -1, -1, -1, -1, -1], dtype=int32),
 0,
 0)

In [85]:
def sent2labels(sent):
    return [label for token, label in sent]

In [86]:
%%time
train, test = train_test_split(data, test_size=0.3, random_state=42)
train_tokens = [sent2features(s) for s in train]
train_labels = [sent2labels(s) for s in train]

test_tokens = [sent2features(s) for s in test]
test_labels = [sent2labels(s) for s in test]

CPU times: user 14min 12s, sys: 3.98 s, total: 14min 16s
Wall time: 14min 19s


In [87]:
print(len(train_tokens)==len(train_labels))
print(len(test_tokens)==len(test_labels))
train1 = [len(x) for x in train_tokens]
train1.sort()
train2 = [len(x) for x in train_labels]
train2.sort()
test1 = [len(x) for x in test_tokens]
test1.sort()
test2 = [len(x) for x in test_labels]
test2.sort()
print(train1==train2)
print(test1==test2)

True
True
True
True


# Baseline

In [88]:
# проставити всім токенам клас False
def classify(token):
    res = []
    for i in range(len(token)):
        res.append(False)     
    return res

In [89]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
test_labels_bin = mlb.fit_transform(test_labels)

In [90]:
predicted = [classify(x) for  x in test_tokens]
print(classification_report(test_labels_bin, mlb.transform(predicted)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2757
           1       0.00      0.00      0.00      2632

   micro avg       1.00      0.51      0.68      5389
   macro avg       0.50      0.50      0.50      5389
weighted avg       0.51      0.51      0.51      5389
 samples avg       1.00      0.52      0.68      5389



/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# CRF 

In [91]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    dep = sent[i][2]
    head = sent[i][3]
    head_pos = sent[i][4]
    lca = sent[i][5]
    left_ch = sent[i][6]
    right_ch = sent[i][7]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'dep': dep,
        'head': head,
        'head_pos': head_pos,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        dep1 = sent[i-1][2]
        head1 = sent[i-1][3]
        head_pos1 = sent[i-1][4]
        
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:dep': dep1,
            '-1:head': head1,
            '-1:head_pos': head_pos1,
            'left_ch': left_ch,
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        dep1 = sent[i+1][2]
        head1 = sent[i+1][3]
        head_pos1 = sent[i+1][4]
        
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:dep': dep1,
            '+1:head': head1,
            '+1:head_pos': head_pos1,
            '+1:lca': sent[lca[i+1]][0],
            'right_ch': right_ch,
        })
    else:
        features['EOS'] = True
        
    if i > 1:
        word1 = sent[i-2][0]
        postag1 = sent[i-2][1]
        dep1 = sent[i-2][2]
        head1 = sent[i-2][3]
        head_pos1 = sent[i-2][4]
        
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:dep': dep1,
            '-2:head': head1,
            '-2:head_pos': head_pos1,
        })
        
    if i < len(sent)-2:
        word1 = sent[i+2][0]
        postag1 = sent[i+2][1]
        dep1 = sent[i+2][2]
        head1 = sent[i+2][3]
        head_pos1 = sent[i+2][4]
        
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:dep': dep1,
            '+2:head': head1,
            '+2:head_pos': head_pos1,
            '+2:lca': sent[lca[i+2]][0],
        })
    
    return features

In [92]:
def sent2features2(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [93]:
%%time
X_train = [sent2features2(s) for s in train_tokens]
y_train = train_labels

X_test = [sent2features2(s) for s in test_tokens]
y_test = test_labels

CPU times: user 890 ms, sys: 12 µs, total: 890 ms
Wall time: 888 ms


In [94]:
def bool_to_string(labels):
    return [str(x) for x in labels]

In [95]:
y_train = [bool_to_string(x) for x in y_train]
y_test = [bool_to_string(x) for x in y_test]

In [97]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 16.1 s, sys: 11.9 ms, total: 16.1 s
Wall time: 16.2 s


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [100]:
eli5.show_weights(crf, top=30)

/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


In [101]:
labels = list(crf.classes_)
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.980319995718865

In [102]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

       False      0.985     0.995     0.990     66066
        True      0.888     0.716     0.793      3476

    accuracy                          0.981     69542
   macro avg      0.937     0.856     0.891     69542
weighted avg      0.980     0.981     0.980     69542



# CRF + perplexity + prob

In [105]:
# модель звідси https://openslr.org/11/
import kenlm
model = kenlm.Model('/tank/liudmyla/4-gram.mmap')
print(model.perplexity('LANGUAGE MODELING IS FUN'))
print(model.perplexity('LANGUAGE MODELING IS'))

5056.315352560162
7988.458056921622


In [106]:
print(model.perplexity('I LIKE BEER'))
print(model.perplexity('I LIKE BEER MORNING'))

112.93413049481077
707.3608785933942


In [300]:
# якщо перплексіті без останнього слова зменшується - ставимо флаг тру для попереднього слова
def shorter_less_perplex(words):
    sentence = ' '.join(words).upper()
    words2 = words[:-1]
    shorter_sentence = ' '.join(words2).upper()
    longer = model.perplexity(sentence)/len(words)
    shorter = model.perplexity(shorter_sentence)/(len(words2))
    return shorter < longer

In [267]:
def two_vs_one(words):
    assert len(words) > 1
    shorter = (model.score(' '.join(words[:-1]).upper()) + model.score(words[-1].upper()))/len(words)
    longer = model.score(' '.join(words).upper())/len(words)
    return shorter < longer

In [331]:
w = "i like beer morning".split()
shorter_less_perplex(w), two_vs_one(w)

(True, True)

In [301]:
import math
def prob(words):
    phrase = ' '.join(words).upper()
    return math.exp(model.score(phrase))

In [359]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    dep = sent[i][2]
    head = sent[i][3]
    head_pos = sent[i][4]
    lca = sent[i][5]
    left_ch = sent[i][6]
    right_ch = sent[i][7]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'dep': dep,
        'head': head,
        'head_pos': head_pos,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        dep1 = sent[i-1][2]
        head1 = sent[i-1][3]
        head_pos1 = sent[i-1][4]
        
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:dep': dep1,
            '-1:head': head1,
            '-1:head_pos': head_pos1,
            'left_ch': left_ch,
        })
    else:
        features['BOS'] = True
        
    for s in (slice(i-3,i+2),slice(i-2,i+2), slice(i-1,i+2)):
        words = sent[s]
        if len(words) > 2:
            features[f'{repr(s)}:shorter-less-perplex'] = shorter_less_perplex([w[0] for w in words])
            features[f'{repr(s)}:two-vs-one'] = two_vs_one([w[0] for w in words])

    for back in (3, 2, 1):
        words = sent[i-back:i+2]
        if len(words) > 2:
            features[f'{back}:shorter-less-perplex'] = shorter_less_perplex([w[0] for w in words])
            features[f'{back}:two-vs-one'] = two_vs_one([w[0] for w in words])


    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        dep1 = sent[i+1][2]
        head1 = sent[i+1][3]
        head_pos1 = sent[i+1][4]
        
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:dep': dep1,
            '+1:head': head1,
            '+1:head_pos': head_pos1,
            '+1:lca': sent[lca[i+1]][0],
            'right_ch': right_ch,
            '+1:prob': prob([sent[i][0],sent[i+1][0]])
        })
    else:
        features['EOS'] = True
        
    if i > 1:
        word1 = sent[i-2][0]
        postag1 = sent[i-2][1]
        dep1 = sent[i-2][2]
        head1 = sent[i-2][3]
        head_pos1 = sent[i-2][4]
        
        
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:dep': dep1,
            '-2:head': head1,
            '-2:head_pos': head_pos1,
        })
        
    if i < len(sent)-2:
        word1 = sent[i+2][0]
        postag1 = sent[i+2][1]
        dep1 = sent[i+2][2]
        head1 = sent[i+2][3]
        head_pos1 = sent[i+2][4]
        
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:dep': dep1,
            '+2:head': head1,
            '+2:head_pos': head_pos1,
            '+2:lca': sent[lca[i+2]][0],
            '+2:prob': prob([sent[i][0],sent[i+1][0],sent[i+2][0]])
        })
    if i > 2:
        word1 = sent[i-3][0]
        postag1 = sent[i-3][1]
        dep1 = sent[i-3][2]
        head1 = sent[i-3][3]
        head_pos1 = sent[i-3][4]
        
        
        features.update({
            '-3:word.lower()': word1.lower(),
            '-3:word.istitle()': word1.istitle(),
            '-3:word.isupper()': word1.isupper(),
            '-3:postag': postag1,
            '-3:dep': dep1,
            '-3:head': head1,
            '-3:head_pos': head_pos1,
        })
    if i < len(sent)-3:
        word1 = sent[i+3][0]
        postag1 = sent[i+3][1]
        dep1 = sent[i+3][2]
        head1 = sent[i+3][3]
        head_pos1 = sent[i+3][4]
        
        features.update({
            '+3:word.lower()': word1.lower(),
            '+3:word.istitle()': word1.istitle(),
            '+3:word.isupper()': word1.isupper(),
            '+3:postag': postag1,
            '+3:dep': dep1,
            '+3:head': head1,
            '+3:head_pos': head_pos1,
            '+3:lca': sent[lca[i+3]][0],
            '+3:prob': prob([sent[i][0],sent[i+1][0],sent[i+2][0],sent[i+3][0]])
        })
    
    return features

In [360]:
train[15]

[['Cold', False],
 ['comfort', True],
 ['Queer', False],
 ['the', False],
 ['whole', False],
 ['atmosphere', False],
 ['of', False],
 ['the', False],
 ['.', False]]

In [364]:
sent2features2(train_tokens[15])[1]

{'bias': 1.0,
 'word.lower()': 'comfort',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'postag': 'NOUN',
 'dep': 'compound',
 'head': 'Queer',
 'head_pos': 'VERB',
 '-1:word.lower()': 'cold',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:postag': 'ADJ',
 '-1:dep': 'amod',
 '-1:head': 'comfort',
 '-1:head_pos': 'NOUN',
 'left_ch': 1,
 'slice(0, 3, None):shorter-less-perplex': True,
 'slice(0, 3, None):two-vs-one': True,
 '1:shorter-less-perplex': True,
 '1:two-vs-one': True,
 '+1:word.lower()': 'queer',
 '+1:word.istitle()': True,
 '+1:word.isupper()': False,
 '+1:postag': 'VERB',
 '+1:dep': 'ROOT',
 '+1:head': 'Queer',
 '+1:head_pos': 'VERB',
 '+1:lca': 'Queer',
 'right_ch': 0,
 '+1:prob': 5.2374711376246165e-06,
 '+2:word.lower()': 'the',
 '+2:word.istitle()': False,
 '+2:word.isupper()': False,
 '+2:postag': 'DET',
 '+2:dep': 'det',
 '+2:head': 'atmosphere',
 '+2:head_pos': 'NOUN',
 '+2:lca': '.',
 '+2:prob': 7.856116438250851e-08,
 '

In [379]:
def sent2features2(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [380]:
%%time
X_train = [sent2features2(s) for s in train_tokens]
y_train = train_labels

X_test = [sent2features2(s) for s in test_tokens]
y_test = test_labels

CPU times: user 9.35 s, sys: 104 ms, total: 9.46 s
Wall time: 9.57 s


In [381]:
def bool_to_string(labels):
    return [str(x) for x in labels]

In [382]:
y_train = [bool_to_string(x) for x in y_train]
y_test = [bool_to_string(x) for x in y_test]

In [383]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 27.4 s, sys: 11.4 ms, total: 27.5 s
Wall time: 27.5 s


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [384]:
eli5.show_weights(crf, top=30)

/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


In [385]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

       False      0.987     0.995     0.991     66066
        True      0.882     0.756     0.814      3476

    accuracy                          0.983     69542
   macro avg      0.935     0.875     0.903     69542
weighted avg      0.982     0.983     0.982     69542



In [386]:
labels = list(crf.classes_)
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.9825528885663546

# Гіпепараметри

In [387]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 12.0min finished


CPU times: user 3min 52s, sys: 20.9 s, total: 4min 13s
Wall time: 12min 29s


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_...
                   iid='deprecated', n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0826a916d0>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f090a4881c0>},
                

In [388]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.23285693208441352, 'c2': 0.1041527090900344}
best CV score: 0.9805927150153408
model size: 2.15M


In [389]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

       False      0.988     0.995     0.991     66066
        True      0.883     0.761     0.817      3476

    accuracy                          0.983     69542
   macro avg      0.935     0.878     0.904     69542
weighted avg      0.982     0.983     0.982     69542



In [390]:
eli5.show_weights(crf, top=30)

/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


# Тестові дані

In [326]:
with open('run-on-test.json') as f:
    hold = json.load(f)

In [391]:
hold_tokens = [sent2features(s) for s in hold]
hold_tokens = [sent2features2(s) for s in hold_tokens]
hold_labels = [sent2labels(s) for s in hold]
hold_labels = [bool_to_string(x) for x in hold_labels]

In [392]:
print(len(hold_tokens)==len(hold_labels))
hold1 = [len(x) for x in hold_tokens]
hold1.sort()
hold2 = [len(x) for x in hold_labels]
hold2.sort()
print(hold1==hold2)

True
True


In [393]:
hold_pred = crf.predict(hold_tokens)
metrics.flat_f1_score(hold_labels, hold_pred, 
                      average='weighted', labels=labels)

0.9731743666169895

In [394]:
print(metrics.flat_classification_report(
    hold_labels, hold_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

       False      0.986     0.986     0.986      4542
        True      0.594     0.594     0.594       155

    accuracy                          0.973      4697
   macro avg      0.790     0.790     0.790      4697
weighted avg      0.973     0.973     0.973      4697



# Висновки 
